Проверьте, встречаются ли в данных повторяющиеся наблюдения, и в качестве ответа укажите их количество. Если они есть, то удалите их из retail.  
Создайте колонку Revenue с суммой покупки, используя колонки Quantity и UnitPrice.

Если пользователь отменил заказ, в начале номера транзакции (InvoiceNo) ставится C (canceled). 
Сколько всего заказов отменили пользователи?

In [2]:
import pandas as pd

retail = pd.read_csv("C:/Users/yagjh/Desktop/Python Files/data_5.csv", encoding="windows-1254", parse_dates=["InvoiceDate"])
retail["Revenue"] = retail["Quantity"] * retail["UnitPrice"]
retail_columns = retail.columns.to_list()
print(retail.shape[0] - retail.drop_duplicates().shape[0], "дубликатов было удалено")
retail = retail.drop_duplicates()
print(retail["InvoiceNo"].str.startswith("C").value_counts()[True], "заказов отменили пользователи")
retail_no_cancel = retail[~retail["InvoiceNo"].str.startswith("C")]
retail

5268 дубликатов было удалено
9251 заказов отменили пользователи


,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,Revenue
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom,15.30
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,20.34
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.0,United Kingdom,22.00
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,20.34
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,20.34
...,...,...,...,...,...,...,...,...,...
541904,581587,22613,PACK OF 20 SPACEBOY NAPKINS,12,2011-12-09 12:50:00,0.85,12680.0,France,10.20
541905,581587,22899,CHILDREN'S APRON DOLLY GIRL,6,2011-12-09 12:50:00,2.10,12680.0,France,12.60
541906,581587,23254,CHILDRENS CUTLERY DOLLY GIRL,4,2011-12-09 12:50:00,4.15,12680.0,France,16.60
541907,581587,23255,CHILDRENS CUTLERY CIRCUS PARADE,4,2011-12-09 12:50:00,4.15,12680.0,France,16.60


Отфильтруйте данные и оставьте в retail только те заказы, где Quantity > 0. В качестве ответа укажите число оставшихся строк.

In [3]:
retail = retail.query("Quantity > 0")
print(retail.shape[0], "строки осталось после фильтрации")

526054 строки осталось после фильтрации


Посчитайте число заказов для каждого пользователя (CustomerID) из Германии (Germany). Оставьте только тех, кто совершил более N транзакций (InvoiceNo), где N – 80% процентиль.  
Запишите полученные id пользователей в germany_top (не весь датафрейм, только id). Идентификатор заказа – InvoiceNo. Для каждого заказа в данных может встречаться более 1 строки.

In [4]:
g_retail = retail.query("Country == 'Germany'").groupby("CustomerID", as_index=False).agg({"InvoiceNo":"count"})
germany_top = g_retail[g_retail["InvoiceNo"] > g_retail["InvoiceNo"].quantile(0.8)]["CustomerID"]
germany_top.head()

3    12471.0
4    12472.0
5    12473.0
6    12474.0
8    12476.0
Name: CustomerID, dtype: float64

Используя объект с id пользователей (germany_top), полученный на предыдущем шаге, отфильтруйте наблюдения и оставьте в данных записи только по интересующим нас юзерам. Результирующий датафрейм запишите в top_retail_germany.


In [5]:
top_retail_germany = retail[retail["CustomerID"].isin(germany_top)]
top_retail_germany.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,Revenue
1109,536527,22809,SET OF 6 T-LIGHTS SANTA,6,2010-12-01 13:04:00,2.95,12662.0,Germany,17.7
1110,536527,84347,ROTATING SILVER ANGELS T-LIGHT HLDR,6,2010-12-01 13:04:00,2.55,12662.0,Germany,15.3
1111,536527,84945,MULTI COLOUR SILVER T-LIGHT HOLDER,12,2010-12-01 13:04:00,0.85,12662.0,Germany,10.2
1112,536527,22242,5 HOOK HANGER MAGIC TOADSTOOL,12,2010-12-01 13:04:00,1.65,12662.0,Germany,19.8
1113,536527,22244,3 HOOK HANGER MAGIC GARDEN,12,2010-12-01 13:04:00,1.95,12662.0,Germany,23.4


Сгруппируйте top_retail_germany по коду товара (StockCode). Какой из продуктов добавляли в корзину чаще всего, кроме POST?  
Note: одним заказом считается единовременная покупка любого количества товара, т.е. без учета Quantity.

In [6]:
best_stock = top_retail_germany.groupby("StockCode", as_index=False).agg({"Quantity":"count"}).nlargest(2, columns="Quantity").iloc[1][0]
print("Товар с кодом", best_stock, "- самый популярный после POST")


Товар с кодом 22326 - самый популярный после POST


Для каждой транзакции (InvoiceNo), посчитайте финальную сумму заказа. В качестве ответа укажите топ-5 (InvoiceNo) по сумме заказа (через запятую с пробелом и в порядке убывания TotalRevenue)

In [7]:
print("5 самых прибыльных товаров:", ", ".join(retail_no_cancel.groupby("InvoiceNo", as_index=False).agg({"Revenue":"sum"}) \
    .nlargest(5, columns="Revenue")["InvoiceNo"].to_list()))

5 самых прибыльных товаров: 581483, 541431, 574941, 576365, 556444
